In [ ]:
import trainHelper
import utils
from mds.lmds import landmarkMDS
from mds.cmds import classicalMDS
from mds.fastmap import fastmap
import numpy
from torch.utils.data import DataLoader
import torch
from torch import nn, Tensor
import lossFunction as lossF
from model.AutoEncoder import AutoEncoder
from model.DynParam import DynParam
from model.Linear import Linear
from model.VAE import VAE
import os.path
import glob

import sys
from importlib import reload
reload(sys.modules['utils']);
reload(sys.modules['trainHelper']);
reload(sys.modules['lossFunction']);

torch.set_default_tensor_type('torch.DoubleTensor')


In [ ]:
sample_space = (1000000, 100)
ss, N, d = 1600, 10, 2

In [ ]:


try:
    euclidean_data1 = utils.load_variable('data/euclidean_data1.pkl')
    euclidean_data2 = utils.load_variable('data/euclidean_data2.pkl')
    
    rand_data1 = utils.load_variable('data/rand_data.pkl')
    rand_data2 = utils.load_variable('data/rand_data.pkl')
    
    if euclidean_data.size() != (ss, 1, N, N):
        print("Updated data for requirement !")
        raise Exception("Previous data not match requirement !")

except:
    euclidean_data1 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    euclidean_data2 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data1 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data2 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    utils.dump_variable(euclidean_data1, 'data/euclidean_data1.pkl')
    utils.dump_variable(euclidean_data2, 'data/euclidean_data2.pkl')

    utils.dump_variable(rand_data1, 'data/rand_data1.pkl')
    utils.dump_variable(rand_data2, 'data/rand_data2.pkl')

In [ ]:
data = torch.stack([
            # euclidean_data1,
            # euclidean_data2,
            rand_data1,
            rand_data2,
        ])
        
data = data.view(ss * 2, 1, N, N)

data = utils.minmax_norm(data, dmin=0)[0]

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

In [ ]:
def MSES(model_dm, target_dm):

    loss = (model_dm - target_dm.view_as(model_dm)) ** 2
    loss = loss.view(loss.size()[0], 1, -1)

    return torch.sum(torch.sum(loss, dim=2) ** 2)

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [16, 32, 64]:

    for i in range(2, 6):

        model_id = "Linear_SGD_" + str(i)+ "_" + str(neuron) + "_distance"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 final_activation=None)

        optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
        
        lossFun = lossF.CoordsToDMLoss(target_dim=(N, 2), 
            lossFun=lossF.DistanceLoss(
                lossFun=MSES
            )
        )

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(3):

            EPOCH = 300
            
            print("Training ", helper.id)

            helper.train(dlr, EPOCH, print_on_each=10)
            helper.backup()

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s")    
            
            print("Test Result: ", 
                            test_model(helper.model, helper.preprocess, test_data, (N, 2)))


In [ ]:
def MSES(model_dm, target_dm):

    loss = (model_dm - target_dm.view_as(model_dm)) ** 2
    loss = loss.view(loss.size()[0], 1, -1)

    return torch.sum(torch.sum(loss, dim=2) ** 2)

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [16, 32, 64]:

    for i in range(2, 6):

        model_id = "AE_SGD_" + str(i)+ "_" + str(neuron) + "_distance"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = AutoEncoder([in_dim, *[neuron for j in range(i)], out_dim],
                 final_activation=None)

        optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
        
        lossFun = lossF.DistanceLoss(
                lossFun=MSES
                )

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(3):

            EPOCH = 300
            
            print("Training ", helper.id)

            helper.train(dlr, EPOCH, print_on_each=10)
            helper.backup()

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s")    
            
            print("Test Result: ", 
                            test_model(helper.model, helper.preprocess, test_data, (N, 2)))


In [ ]:
def MSES(model_dm, target_dm):

    loss = (model_dm - target_dm.view_as(model_dm)) ** 2
    loss = loss.view(loss.size()[0], 1, -1)

    return torch.sum(torch.sum(loss, dim=2) ** 2)

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [16, 32, 64]:

    for i in range(2, 6):

        model_id = "VAE_SGD_" + str(i)+ "_" + str(neuron) + "_distance"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = AutoEncoder([in_dim, *[neuron for j in range(i)], out_dim],
                 final_activation=None)

        optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
        
        lossFun = lossF.DistanceLoss(
                lossFun=MSES
                )

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(3):

            EPOCH = 300
            
            print("Training ", helper.id)

            helper.train(dlr, EPOCH, print_on_each=10)
            helper.backup()

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s")    
            
            print("Test Result: ", 
                            test_model(helper.model, helper.preprocess, test_data, (N, 2)))


In [ ]:

utils.plot_records(
    helper.records[
        ['loss_mean', 'loss_max', 'loss_min']
    ].to_dict(orient='list'), 
    helper.epoch, value_label='loss')

utils.plot_records(
    helper.records[['train_time']].to_dict(orient='list'), 
    helper.epoch, value_label='train_time')

